In [1]:
def Mod(a, b):
    return a%b

def mod_binary(k, g, p):
    y=1
    k = str(bin(k))[2:]
    for i in k:
        if i=='1':
            y = Mod(Mod(y**2, p)*g, p)
            # print(y)
        else:
            y = Mod(y**2, p)
            # print(y)
    return y

def ex_euclid(a, b):
    A = [a, b]
    X = [1, 0]
    Y = [0, 1]
    i=1
    while A[-1]!=0:
        q = A[i-1]//A[i]
        a_next = A[i-1]-A[i]*q
        x_next = X[i-1]-q*X[i]
        y_next = Y[i-1]-q*Y[i]
        A.append(a_next)
        X.append(x_next)
        Y.append(y_next)
        i+=1
    
    ans = []
    ans.append((X[i-1], Y[i-1]))
    d = A[i-1]
    x = X[i-1]
    y = Y[i-1]
    return x, y, d

def inv(a, n):
    x, y, d = ex_euclid(a, n)
    inv = x%n
    # print("inv =", inv)
    # print("inv*a mod n =", (inv*a)%n)
    return inv

In [9]:
import hashlib, math
# ハッシュ関数Shake128によるハッシュ値の計算
# Input: メッセージ m (通常の文字列), ベースポイントの位数 l (整数)
# Output: shake128による m のハッシュ値 (16進文字列)
def shake128(m, l):
    return hashlib.shake_128(m.encode('utf-8')).hexdigest(math.ceil(l.bit_length()/8)-1)

def dsa_sign_gen_key(p, g, l, sk):
    #Input: 法 mod：p, ベースポイント base point：g, ベースポイントの位数 base point number position：ℓ, 秘密鍵secret key：sk
    #Output: 公開鍵 public key: pk = gsk mod p
    return mod_binary(sk, g, p)

def dsa_sign_gen(m, g, l, r, p, sk):
    u = mod_binary(r, g, p) % l
    m_prime = shake128(str(m), l)
    v = (inv(r, l)*(int(m_prime, 16) + sk*u)) % l
    return (u, v)

def dsa_sign_verify(m, sigma, l, g, p, pk):
    u, v = sigma
    m_prime = shake128(str(m), l)
    v_inv = inv(v, l)
    u_prime = (mod_binary(int(m_prime, 16)*v_inv, g, p) * mod_binary(u*v_inv, pk, p)) % p % l
    if u_prime == u:
        return 'OK'
    else:
        return 'NG'

def elgamal_enc(m, g, p, r, pk):
    u = mod_binary(r, g, p)
    c = Mod(mod_binary(r, pk, p)*int(m), p)
    return (u, c)

def elgamal_dec(u, c, p, sk):
    m = Mod(c*inv(mod_binary(sk, u, p), p), p)
    return m

In [7]:
def hybrid_key_gen(p, g, sk):
    #Input: 法 mod：p, ベースポイント base point：g, 秘密鍵 secret key：sk
    #Output: 公開鍵 public key: pk = gsk mod p
    return mod_binary(sk, g, p)

def hybrid_enc_sign_gen(m, g, l, r, p, sk, pk):
    # Input: メッセージ (通常の文字列) message (string)：m, 秘密鍵 secret key：sk, 公開鍵 public key：pk, ベースポイン
    # ト base point：g, ベースポイントの位数 base point number position：ℓ, 乱数 (整数) random number (integer)：
    # r, 法 mod：p
    # Output: m の暗号文 (整数の配列) m cyphertext(integer array)：[c1 = gr, c2 = pkrm], 署名 (整数の配列) signa-
    # ture(integer array)：[u, v]
    return elgamal_enc(m, g, p, r, pk), dsa_sign_gen(m, g, l, r, p, sk)

def hybrid_enc_sign_verify(C, sigma, sk, l, g, p, pk):
    # Input: m の暗号文 ciphertext of m：C = [c1, c2], m の署名 signature of m：sigma=[u, v], ベースポイント base point：
    # g, ベースポイントの位数 base point number position：ℓ, 公開鍵 public key：pk, 秘密鍵 secret key：sk, 法 mod：p
    # Output: C の復号結果 m, 署名が正しければ True, 誤っていれば False C decryption of m, If signatures are correct
    # True, if wrong False.
    m = elgamal_dec(C[0], C[1], p, sk)
    return m, dsa_sign_verify(m, sigma, l, g, p, pk)

In [10]:
sk = 172350919665451459123451714499640833062234544321
p4 = 141108755332974711606815218263958123381184588212061018448136404826965889433079453789166218230378522228564058127860367190611065605375025546257531489369344062782521806978218808940091447658298351853603237069980597505163602473095615670998464391197300372933147772009493823037167642459378452731092557170909406945309
g4 = 79207621787760038235763239269746451281552097558625763440050213854787240633084667257397421010854631623596917364929357681934505810865796708268321894884518347711092708958596829555918931536779252059763083320084867242870421484137196365244257388686997557137455046446997809953054632950185678637137955632299024284915
l4 = 1362115923099293242369922261305212343561846087883
m="123750811111151459123451714499640333335555544444"
r=123456789123456789123456789123456789123456789123

pk = hybrid_key_gen(p4, g4, sk)
print("pk =", pk)
C, sigma = hybrid_enc_sign_gen(m, g4, l4, r, p4, sk, pk)
print("C =", C)
print("sigma =", sigma)
m_dec, verify = hybrid_enc_sign_verify(C, sigma, sk, l4, g4, p4, pk)
print("m_dec =", m)
print("verify =", verify)

pk = 115663660594644880912588805079680708156039222725243914381461795099358616191213564389068683001245618651376169359590787727427879769324617075262467928829932624729289740823959396773417044649900708197166760276052910230998436195192942588635192401623624735403251438866691781501953206435036413485250914737541760231041
C = (1363808330698149239544835062287340367964290271461941596678126312354987971893999754816126167000860796644450549359642698944046472337842217652254316490529730333744436220453816538286171824432231291826033754798476109749048209131674858725237000742469868855972120446274563247542084169537026431581380584720197069919, 51505933080376757025064602732273539655716311304987842747858983611311354905745894102463864520120554456749219965246546248374976790648840213629393253525604885361846566066932558151975693529497796013793300555089104186335029602863339471368357489926655490042076705341321731152819158050115274427217700393419676789118)
sigma = (1198642514289996608651292588855800695584441096609, 1